Activity 2.6 -- Working with COVID 19 and World Bank Data
=========================================================

In this activity, you will explore relationships between various World
Bank indicators for countries and their corresponding COVID death rates.
First you need to download data on COVID-19 (see links and instructions
below) and the selected indicators from the Open World Bank data
available at <https://data.worldbank.org>.

**COVID data set source:** <https://coviddata.github.io/coviddata/#csvs>

**Tasks.** Use pandas and dfply to perform each of the following.

1.  Download the raw **time\_series\_covid19\_confirmed\_global.csv**
    dataset.

2.  Inspect the data and discuss the need to reshape. 

In [12]:
import pandas as pd
from dfply import *
from more_dfply import *

In [9]:
confirmed_global = pd.read_csv('data/time_series_covid19_confirmed_global.csv')

In [10]:
confirmed_global.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/3/22,9/4/22,9/5/22,9/6/22,9/7/22,9/8/22,9/9/22,9/10/22,9/11/22,9/12/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,193912,194163,194355,194614,195012,195298,195471,195631,195925,196182
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,330062,330193,330221,330283,330516,330687,330842,330948,331036,331053
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270426,270443,270461,270476,270489,270507,270522,270532,270539,270551
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,46027,46027,46027,46027,46113,46113,46113,46113,46113,46113
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,102636,102636,102636,102636,102636,102636,103131,103131,103131,103131


> **Your discussion:**

The dates contain information about the rows and thus should be stored in rows, not columns. The dates need to be stacked here and changed to type date.

3.  Write a single pipe that reshapes the data, sets the dtype of the date column, and extracts various date parts.
    1. To change the `dtype` of the date column, `date = X.date.astype('datetime64')`
    2. To extract the year and month, use the `X.date.dt.year` and `X.date.dt.month` attributes. This will need to happen in a separate `mutate` 

In [64]:
confirmed_global_reshaped = (confirmed_global
    >> gather("Date", "Cases", columns_from("1/22/20"))
    >> mutate(Date = X.Date.astype('datetime64'))
    >> mutate(Year = X.Date.dt.year,
              Month = X.Date.dt.month
            )
    >> rename(country = X["Country/Region"])
)

In [65]:
confirmed_global_reshaped.head()

,Province/State,country,Lat,Long,Date,Cases,Year,Month
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,2020,1
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,2020,1
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,2020,1
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,2020,1
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,2020,1


### World Bank Links Development Indicators

<https://databank.worldbank.org/source/world-development-indicators>

#### Constructing a data set.

First you need to construct a data set as follows

1.  Expand the Country tab and select all.

<img src="./img/media/image1.png" width="300">

2.  Click on the Series tab, search for *Health* and select the
    following indicators. **Feel free to add additional indicators!**

<img src="img/media/image2.png" width="300">

3.  Click on the Time tab and select 2020 and 2021.

4.  Click apply changes in the floating dialog.

<img src="img/media/image3.png" width="300">

5.  Select CSV from the Download Options button and save the data folder

<img src="img/media/image4.png" width="100">

#### Tasks

Use pandas and dfply to perform each of the following.

1.  Inspect the World Bank data and discuss the need to reshape. 

**Hints:** 

* You should apply `fix_names` from `more_dfply` to clean up the column names.
* This table needs to be reshaped twice




In [37]:
world_bank_data = pd.read_csv("data/849ec2a4-2293-477b-9e04-7bd97686e927_Data.csv")

In [46]:
?spread

Signature:   spread(*args, **kwargs)
Type:        pipe
String form: <dfply.base.pipe object at 0x7f7a58a27e50>
File:        ~/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/base.py
Docstring:  
Transforms a "long" DataFrame into a "wide" format using a key and value
column.

If you have a mixed datatype column in your long-format DataFrame then the
default behavior is for the spread columns to be of type `object`, or
string. If you want to try to convert dtypes when spreading, you can set
the convert keyword argument in spread to True.

Args:
    key (str, int, or symbolic): Label for the key column.
    values (str, int, or symbolic): Label for the values column.

Kwargs:
    convert (bool): Boolean indicating whether or not to try and convert
        the spread columns to more appropriate data types.


Example:
    widened = elongated >> spread(X.variable, X.value)
    widened >> head(5)

        _ID carat clarity color        cut depth price table     x     y   

In [53]:
world_bank_data_reshaped.columns

Index([                                                                 'Country Name',
                                                                        'Country Code',
                                                                         'Series Code',
                                                                                 '_ID',
                                                                                'year',
                                                                                   nan,
                                               'Current health expenditure (% of GDP)',
                                 'Current health expenditure per capita (current US$)',
                           'Domestic general government health expenditure (% of GDP)',
             'Domestic general government health expenditure per capita (current US$)',
               'Domestic private health expenditure (% of current health expenditure)',
       'Domestic private health 

In [62]:
world_bank_data_reshaped = (world_bank_data
    >> gather("year", "data", columns_from("2018 [YR2018]"), add_id = True)
    >> rename(series_name = X["Series Name"],
              country = X["Country Name"]
    )
    >> spread(X.series_name, X.data)
)

In [63]:
world_bank_data_reshaped.head()

,country,Country Code,Series Code,_ID,year,NaN,Current health expenditure (% of GDP),Current health expenditure per capita (current US$),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure per capita (current US$),Domestic private health expenditure (% of current health expenditure),"Domestic private health expenditure per capita, PPP (current international $)",External health expenditure (% of current health expenditure)
0,Afghanistan,AFG,SH.XPD.CHEX.GD.ZS,0,2018 [YR2018],NaN,14.12674332,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,SH.XPD.CHEX.GD.ZS,0,2019 [YR2019],NaN,13.24220181,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,SH.XPD.CHEX.PC.CD,1,2018 [YR2018],NaN,NaN,69.99860382,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,SH.XPD.CHEX.PC.CD,1,2019 [YR2019],NaN,NaN,65.80603027,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,SH.XPD.EHEX.CH.ZS,6,2018 [YR2018],NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.79253006


> **Your discussion:**

2.  Write a single pipe that reshapes the data, sets the dtype of the date column, and extracts various date parts.
    1. You can use the `replace` method to clean up the year column.  See lecture 3.1 for details.

In [4]:
# your code here

### Investigate joining on country

Before we can proceed, we need to make sure that the columns used to join the data--namely the country--actually match.  Do this by

1. For each table, select just the country columns and make sure the column names match.
2. Perform a full outer join and filter on rows that didn't match (i.e. with a missing value in either column).
3. Determine any transformations needed to make the entries match.
4. Transform each of the original table as need (column names and/or problematic entries.

In [5]:
# Your code here

## Join and visualize 

Finally, you should use pandas and dfply to join these two data sets together, then create some interesting visualization using seaborn.

In [6]:
# Your code here

### Deliverables
To complete this part of the activity, you need to submit the following.

1.  A link to this notebook including all discussion and code requests
    above.

2.  A csv file containing your final dataset. **Hint.** You can use the
    to\_csv method on the final data frame.

In [7]:
# Code for writing the data here